In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import random
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [2]:
data = pd.read_csv('retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [3]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [4]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    # Подсказка: необходимо модифицировать функцию random_recommendation()
    items = items_weights['item_id'].values
    weights = items_weights['weight'].values
    recs = random.choices(population=items, weights=weights, k=n)
    
    return recs

In [5]:
%%time

sum_sales__values_all_items = sum(data.sales_value)
list_items_weights = data.sales_value / sum_sales__values_all_items
items_weights = pd.DataFrame({'item_id': pd.Series(data.item_id.values), 'weight':list_items_weights}, columns=['item_id', 'weight'])

Wall time: 192 ms


In [6]:
result_weighted_random = data_test.groupby('user_id')['item_id'].unique().reset_index()
result_weighted_random.columns=['user_id', 'actual']
result_weighted_random.head()

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107..."
3,7,"[840386, 889774, 898068, 909714, 929067, 95347..."
4,8,"[835098, 872137, 910439, 924610, 992977, 10412..."


In [ ]:
%%time
result_weighted_random['weighted_random_recommendation'] = result_weighted_random['user_id'].apply(lambda x: weighted_random_recommendation(items_weights, n=5))

In [ ]:
result_weighted_random.head()

In [ ]:
result = pd.read_csv('predictions_basic.csv')
result.head()

In [ ]:
result['weighted_random_recommendation'] = result_weighted_random['weighted_random_recommendation']

In [ ]:
result.head()

### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [ ]:
def precision_at_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    flags = np.isin(bought_list, recommended_list)
    
    precision = (flags.sum() / len(recommended_list))
    
    return precision

In [ ]:
dict_metrics = {'random_recommendation': precision_at_k(result.random_recommendation, result.actual, 5),
                'popular_recommendation': precision_at_k(result.popular_recommendation, result.actual, 5),
                'itemitem': precision_at_k(result.itemitem, result.actual, 5),
                'cosine': precision_at_k(result.cosine, result.actual, 5),
                'tfidf': precision_at_k(result.tfidf, result.actual, 5),
                'own_purchases': precision_at_k(result.own_purchases, result.actual, 5),
                'weighted_random_recommendation': precision_at_k(result.weighted_random_recommendation, result.actual, 5),}

In [ ]:
dict_metrics

Наилучший результат дал алгоритм own purchases